In [1]:
import pymongo
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time
from datetime import datetime 
from datetime import timedelta
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import json
import requests
from pymongo.errors import BulkWriteError
from pprint import pprint
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from itertools import chain
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
header = {'Accept' : 'application/json',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
            'Accept-Encoding':'gzip, deflate, br',
            'Accept-Language' :  'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6',
            'Connection': 'keep-alive',
            'Sec-Fetch-Site':'same-site',
            'Sec-Fetch-Mode':'cors',
            'Sec-Fetch-Dest':'empty'}


stock_name = 'META'                # this name will come from flask frontend
main_data = []                    #  All data will store here
api_next_list = []


time = 24
current_time = datetime.now()
time = current_time - timedelta(hours=time)


#Initializing the request and converts it into json
url = f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22'  
request_success = requests.get(url,headers=header)   
json_data_dict = request_success.json()


#Taking all the usefull data from the json data
usefull_data = (json_data_dict['messages']) 

#picking the next api address for further scrapping
next_api_first = json_data_dict['cursor']['max']
next_api_first = str(next_api_first)
next_api_first = '&max='+next_api_first



# Retrieving the last date from twit to break the first loop
last_twit_date_1 = (json_data_dict['messages'][-1])
last_twit_date_1 = last_twit_date_1['created_at']
last_twit_date_1 = re.sub(pattern = "[T-Z]+",repl = ' ',string=last_twit_date_1) # eliminating alphabets
last_twit_date_1 = datetime.strptime(last_twit_date_1, "%Y-%m-%d  %H:%M:%S ")  # changing datatype



#Looping in the usefull_data
for data in usefull_data:
    # Getting the twit date from  usefull_data and cleaning is done
    twit_date = data['created_at']
    twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
    twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   
    
    if twit_date > time:
        twit = data['body']
        signal = data['entities']['sentiment']
        if signal == None:
            signal = 'Null'
        else :
            signal = signal['basic']
        stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal': signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
        main_data.append(stock_twits_data)
    if twit_date < time:
        if len(main_data) == 0:
            print('No twits found')
        else:
            print('twits are there')
        break
    try:
        if time < last_twit_date_1.strptime('%H:%M:%S'):
            break
    except:
        pass

#########################################################################################################################################    
while (True):
    loop_break = []
    url =  f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22{next_api_first}'
    request_success = requests.get(url,headers=header)   
    json_data_dict = request_success.json() 
    next_api_first = json_data_dict['cursor']['max']
    next_api_first = str(next_api_first)
    next_api_first = '&max='+next_api_first
    usefull_data = (json_data_dict['messages'])
    for data in usefull_data:
        twit_date = data['created_at']
        twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
        twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   
        if twit_date > time:
            twit = data['body']
            signal = data['entities']['sentiment']
            if signal == None:
                signal = 'Null'
            else :
                signal = signal['basic']
            stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal':signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
            main_data.append(stock_twits_data)
            
    if twit_date < time:
        print('while loop break')
        break
        
############################################# Cleaning Code Begins ########################################################

df = pd.DataFrame(main_data)
pd.set_option('display.max_colwidth', None)

##Removing Links from dataframe
print('---------------------------------Links------------------------------------')
df['Twits'] = df['Twits'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


## Lowercase the whole twits column
print('---------------------------------lowercase------------------------------------')
df['Twits'] = df['Twits'].apply(lambda x: x.lower())


## Eliminating the pumctuation 
print('----------------------------eliminate punctuation-------------------------------')
df['Twits'] = df['Twits'].str.replace('[^\w\s]',' ',regex=True)


##Eliminating stopwords 
def remove_stopwords(x):
    return " ".join([word for word in str(x).split() if word not in stop_words])
stop_words = set(stopwords.words('english'))
print('----------------------------removing stopwords-------------------------------')
df['Twits'] = df['Twits'].apply(lambda x : remove_stopwords(x))


##Eliminating Emoji from dataframe
print('----------------------------removing emojis-------------------------------')
df['Twits'] = df['Twits'].apply(lambda s: emoji.replace_emoji(s, ''))

# Stemming on dataframe
print('----------------------------stemming is done-------------------------------')
stemmer = PorterStemmer()
df['Twits'] = df['Twits'].str.split()
df['Twits'] = df['Twits'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))

# Removing number str from column twits
print('----------------------------removing number is done-------------------------------')
df['Twits'] = df['Twits'].replace('\d+', '', regex=True)

# Removing the words which contain only 1 letter
print('----------------------------removing 1 letter word is done-------------------------------')
df['Twits'] = df['Twits'].str.split()
for text in df['Twits']:
    for word in text:
        if len(word) == 1:
            text.remove(word)
df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))

while loop break
---------------------------------Links------------------------------------
---------------------------------lowercase------------------------------------
----------------------------eliminate punctuation-------------------------------
----------------------------removing stopwords-------------------------------
----------------------------removing emojis-------------------------------
----------------------------stemming is done-------------------------------
----------------------------removing number is done-------------------------------
----------------------------removing 1 letter word is done-------------------------------


In [8]:
main_data

[{'stock_name': 'META',
  'Date': datetime.datetime(2023, 10, 19, 13, 24, 22),
  'Twits': '$META puts getting wrecked lol',
  'Signal': 'Bullish',
  'Cleaning_status': 'pending',
  'Cleaning_status_lemmatization': 'pending'},
 {'stock_name': 'META',
  'Date': datetime.datetime(2023, 10, 19, 13, 11, 47),
  'Twits': '$META why is this dumpster stock so high, this needs to drop $SPY $QQQ',
  'Signal': 'Bearish',
  'Cleaning_status': 'pending',
  'Cleaning_status_lemmatization': 'pending'},
 {'stock_name': 'META',
  'Date': datetime.datetime(2023, 10, 19, 13, 7, 13),
  'Twits': '$META Can we go up to $330.00 and flush it back down to $314.87 by EOD Friday.  Doubt it, but that&#39;s what it did last week, so let&#39;s find out.  Good luck.',
  'Signal': 'Bearish',
  'Cleaning_status': 'pending',
  'Cleaning_status_lemmatization': 'pending'},
 {'stock_name': 'META',
  'Date': datetime.datetime(2023, 10, 19, 12, 57, 59),
  'Twits': '$META better sell now',
  'Signal': 'Bearish',
  'Cleaning_s

In [58]:
df = pd.DataFrame(main_data)
pd.set_option('display.max_colwidth', None)

##Removing Links from dataframe
print('---------------------------------Links------------------------------------')
df['Twits'] = df['Twits'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


## Lowercase the whole twits column
print('---------------------------------lowercase------------------------------------')
df['Twits'] = df['Twits'].apply(lambda x: x.lower())


## Eliminating the pumctuation 
print('----------------------------eliminate punctuation-------------------------------')
df['Twits'] = df['Twits'].str.replace('[^\w\s]',' ',regex=True)


##Eliminating stopwords 
def remove_stopwords(x):
    return " ".join([word for word in str(x).split() if word not in stop_words])
stop_words = set(stopwords.words('english'))
print('----------------------------removing stopwords-------------------------------')
df['Twits'] = df['Twits'].apply(lambda x : remove_stopwords(x))


##Eliminating Emoji from dataframe
print('----------------------------removing emojis-------------------------------')
df['Twits'] = df['Twits'].apply(lambda s: emoji.replace_emoji(s, ''))

# Stemming on dataframe
print('----------------------------stemming is done-------------------------------')
stemmer = PorterStemmer()
df['Twits'] = df['Twits'].str.split()
df['Twits'] = df['Twits'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))

# Removing number str from column twits
print('----------------------------removing number is done-------------------------------')
df['Twits'] = df['Twits'].replace('\d+', '', regex=True)

# Removing the words which contain only 1 letter
print('----------------------------removing 1 letter word is done-------------------------------')
df['Twits'] = df['Twits'].str.split()
for text in df['Twits']:
    for word in text:
        if len(word) == 1:
            text.remove(word)
df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))


---------------------------------Links------------------------------------
---------------------------------lowercase------------------------------------
----------------------------eliminate punctuation-------------------------------
----------------------------removing stopwords-------------------------------
----------------------------removing emojis-------------------------------
----------------------------stemming is done-------------------------------
----------------------------removing number is done-------------------------------
----------------------------removing 1 letter word is done-------------------------------


In [59]:
df

,stock_name,Date,Twits,Signal,Cleaning_status,Cleaning_status_lemmatization
0,META,2023-10-19 13:24:22,meta put get wreck lol,Bullish,pending,pending
1,META,2023-10-19 13:11:47,meta dumpster stock high need drop spi qqq,Bearish,pending,pending
2,META,2023-10-19 13:07:13,meta go flush back eod friday doubt last week let find good luck,Bearish,pending,pending
3,META,2023-10-19 12:57:59,meta better sell,Bearish,pending,pending
4,META,2023-10-19 12:56:47,qqq tsla amzn meta wait tesla turnaround open,Bullish,pending,pending
...,...,...,...,...,...,...
83,META,2023-10-18 19:18:14,qqq market crash incom spi aapl tsla meta,Bearish,pending,pending
84,META,2023-10-18 19:15:45,meta fund global genocid indigen peopl glwt,Bearish,pending,pending
85,META,2023-10-18 19:02:36,let play game guess ticker play buy low got buy fill timestamp gain skeptic buy low perfect level revers trade voo nflx arkk meta goog,Null,pending,pending
86,META,2023-10-18 19:01:20,today call spx meta qqq,Null,pending,pending


In [9]:
print(len(main_data))

88


In [ ]:
import datetime
dt = datetime.datetime(2023, 4, 5, 23, 12, 36)
times = dt.strftime('%X') 
print(times)
print(type(times))